In [76]:
!playwright install
!pip install chromadb
!pip install streamlit


Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.11/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

In [77]:

import google.generativeai as genai
import os
!pip install playwright
import playwright

Setup Gemini API


In [78]:
# --- Step 1: Setup Gemini API ---
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
model = genai.GenerativeModel('gemini-1.5-flash')

Web Scraping

In [79]:
#--- Step 2: web scraping
from playwright.async_api import async_playwright
import asyncio

async def fetch_url_content(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch() # You can choose other browsers like firefox or webkit
        page = await browser.new_page()
        await page.goto(url)
        content = await page.inner_text("body")
        await browser.close()
        return content

# Example usage:
target_url = "https://en.wikisource.org/wiki/The_Gates_of_Morning/Book_1/Chapter_1"



In [80]:
#--- Step 3: ai-driven spinning using gemini
def spin_with_gemini(text):
  ai_writer_prompt = f"""
You are an expert book rewriter. Given the following chapter content, rewrite it in a modern, engaging tone for contemporary readers.

Chapter:
{text}
"""
  response=model.generate_content(ai_writer_prompt)
  return response.text




In [81]:
def save_to_file(content, filename="spun_chapter.md"):
    with open(filename, "w", encoding="utf-8") as f:
        f.write(content)

In [82]:
async def main():
    url = "https://en.wikisource.org/wiki/The_Gates_of_Morning/Book_1/Chapter_1"
    print("[+] Fetching chapter content and screenshot...")
    raw_text = await fetch_url_content(url)
    save_to_file(raw_text, "original_chapter.md")

    print("[+] Spinning with Gemini...")
    spun_text = spin_with_gemini(raw_text)
    save_to_file(spun_text, "spun_chapter.md")

    print("[✓] Done. Saved:")
    print("- Screenshot: chapter.png")
    print("- Original: original_chapter.md")
    print("- Spun: spun_chapter.md")

# --- Entry Point ---
await(main())

[+] Fetching chapter content and screenshot...
[+] Spinning with Gemini...


2025-06-22 17:11:47.491 200 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 12618.96ms


[✓] Done. Saved:
- Screenshot: chapter.png
- Original: original_chapter.md
- Spun: spun_chapter.md


In [83]:
import chromadb
import uuid
from datetime import datetime


client = chromadb.Client()
collection = client.get_or_create_collection(name="books")

def save_version_to_chromadb(text, chapter=1, status="spun"):
    version_id = str(uuid.uuid4())
    metadata = {
        "chapter": chapter,
        "status": status,
        "timestamp": datetime.utcnow().isoformat()
    }

    collection.add(
        documents=[text],
        metadatas=[metadata],
        ids=[version_id]
    )
    print(f"[✓] Stored in ChromaDB: {status} | ID: {version_id}")


In [84]:
def semantic_search_chapter(query_text):
    results = collection.query(
        query_texts=[query_text],
        n_results=3
    )
    return results['documents']


In [85]:
%%writefile app.py
import streamlit as st

# Load content
def load_content(file):
    with open(file, "r", encoding="utf-8") as f:
        return f.read()

st.title("AI-Spun Chapter Review")

original = load_content("original_chapter.md")
spun = load_content("spun_chapter.md")

tab1, tab2 = st.tabs(["📘 Original", "📝 Spun Version"])

with tab1:
    st.subheader("Original Chapter")
    st.text_area("Original Content", original, height=400)

with tab2:
    st.subheader("AI-Spun Chapter")
    edited_text = st.text_area("Review and Edit", spun, height=400)

    if st.button("💾 Save Reviewed Version"):
        with open("final_reviewed_chapter.md", "w", encoding="utf-8") as f:
            f.write(edited_text)
        st.success("Reviewed version saved.")


Overwriting app.py


In [86]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧
up to date, audited 23 packages in 979ms
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠧

In [87]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

104.196.197.208
⠙your url is: https://ripe-icons-stare.loca.lt


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))


Password/Enpoint IP for localtunnel is: 104.196.197.208
